In [1]:
import pandas as pd
from IPython.display import display

# food.csv row values represent 100g worth of the row's food
food = pd.read_csv("food.csv")
food_renamed_columns = {
    'Category': 'category',
    'Description': 'description',
    'Nutrient Data Bank Number': 'nut_db_num',
    'Data.Alpha Carotene': 'alpha_carotene',
    'Data.Ash': 'ash',
    'Data.Beta Carotene': 'beta_carotene',
    'Data.Beta Cryptoxanthin': 'beta_cryptoxanthin',
    'Data.Carbohydrate': 'carbohydrate',
    'Data.Cholesterol': 'cholesterol',
    'Data.Choline': 'choline',
    'Data.Fiber': 'fiber',
    'Data.Kilocalories': 'calories',
    'Data.Lutein and Zeaxanthin': 'lutein_and_zeaxanthin',
    'Data.Lycopene': 'lycopene',
    'Data.Manganese': 'manganese',
    'Data.Niacin': 'niacin',
    'Data.Pantothenic Acid': 'pantothenic_acid',
    'Data.Protein': 'protein',
    'Data.Refuse Percentage': 'pct_refuse',
    'Data.Retinol': 'retinol',
    'Data.Riboflavin': 'riboflavin',
    'Data.Selenium': 'selenium',
    'Data.Sugar Total': 'sugar',
    'Data.Thiamin': 'thiamin',
    'Data.Water': 'water',
    'Data.Fat.Monosaturated Fat': 'fat_monounsaturated',
    'Data.Fat.Polysaturated Fat': 'fat_polyunsaturated',
    'Data.Fat.Saturated Fat': 'fat_saturated',
    'Data.Fat.Total Lipid': 'fat_total',
    'Data.Household Weights.1st Household Weight': 'first_weight',
    'Data.Household Weights.1st Household Weight Description': 'first_weight_desc',
    'Data.Household Weights.2nd Household Weight': 'second_weight',
    'Data.Household Weights.2nd Household Weight Description': 'second_weight_desc',
    'Data.Major Minerals.Calcium': 'calcium',
    'Data.Major Minerals.Copper': 'copper',
    'Data.Major Minerals.Iron': 'iron',
    'Data.Major Minerals.Magnesium': 'magnesium',
    'Data.Major Minerals.Phosphorus': 'phosphorus',
    'Data.Major Minerals.Potassium': 'potassium',
    'Data.Major Minerals.Sodium': 'sodium',
    'Data.Major Minerals.Zinc': 'zinc',
    'Data.Vitamins.Vitamin A - IU': 'v_a_iu',
    'Data.Vitamins.Vitamin A - RAE': 'v_a_rae',
    'Data.Vitamins.Vitamin B12': 'v_b12',
    'Data.Vitamins.Vitamin B6': 'v_b6',
    'Data.Vitamins.Vitamin C': 'v_c',
    'Data.Vitamins.Vitamin E': 'v_e',
    'Data.Vitamins.Vitamin K': 'v_k',
}
food = food.rename(columns=food_renamed_columns)
food["calories_per_gram"] = food["calories"] / 100

rdi = pd.read_csv("rdi.csv")
micronutrients = rdi[rdi.nutrient.isin(food)].nutrient

POUND_IN_GRAMS = 453.592

CATEGORY_EXCLUDES = [
    "ALLSPICE",
    "BABYFOOD",
    "BEVERAGE",
    "CELERY FLAKES",
    "CEREALS RTE",
    "CEREALS",
    "CHERVIL",
    "CHILD FORMULA",
    "CHIVES",
    "CINNAMON",
    "CLOVES",
    "COCOA MIX",
    "CORIANDER LEAF",
    "DESSERTS",
    "DILL WEED",
    "Form Bar",
    "FORM BAR",
    "FORM",
    "INF FORM",
    "LEAVENING AGENTS",
    "LEMONADE",
    "MARGARINE-LIKE SPRD",
    "MARJORAM",
    "MUSTARD SEED",
    "No Category",
    "OIL",
    "PARSLEY",
    "PEPPER",
    "PUDDINGS",
    "ROSEMARY",
    "SAGE",
    "SAVORY",
    "SESAME BUTTER",
    "SEASAME FLOUR",
    "SEASAME MEAL",
    "SESAME SEEDS",
    "SISYMBRIUM SP. SEEDS",
    "SNACKS",
    "SPEARMINT",
    "SPICES",
    "TURMERIC",
    "VANILLA EXTRACT",
    "VITASOY USA",
    "WHEY",
]

# ~ is bitwise negate, functionally this is "give me all data frames that are not in the category excludes list"
food = food[~food.category.isin(CATEGORY_EXCLUDES)]
# Exclude any category with DRK in it -> some kind of BS "drink"
food = food[~food.category.str.contains("DRK")]
# Maybe powdered milk is nutritionally amazing, I don't care
food = food[~food.description.str.contains("MILK,DRY")]

# Create some derived columns:
# air_quality["london_mg_per_cubic"] = air_quality["station_london"] * 1.882
for nutrient in micronutrients:
    food["{}_pct_rdi".format(nutrient)] = (food[nutrient] / rdi[rdi.nutrient == nutrient].rdi.values[0]) * 100
    food["{}_grams_100_pct_rdi".format(nutrient)] = (100.0 / food["{}_pct_rdi".format(nutrient)]) * 100
    food["{}_calories_100_pct_rdi".format(nutrient)] = food["{}_grams_100_pct_rdi".format(nutrient)] * food["calories_per_gram"]

food["total_pct_rdi"] = food["calcium_pct_rdi"] + food["choline_pct_rdi"] + food["copper_pct_rdi"] + food["iron_pct_rdi"] + food["magnesium_pct_rdi"] + food["manganese_pct_rdi"] + food["niacin_pct_rdi"] + food["pantothenic_acid_pct_rdi"] + food["phosphorus_pct_rdi"] + food["potassium_pct_rdi"] + food["riboflavin_pct_rdi"] + food["selenium_pct_rdi"] + food["thiamin_pct_rdi"] + food["v_a_rae_pct_rdi"] + food["v_b12_pct_rdi"] + food["v_b6_pct_rdi"] + food["v_c_pct_rdi"] + food["v_e_pct_rdi"] + food["v_k_pct_rdi"] + food["zinc_pct_rdi"]
food["pct_rdi_per_calorie"] = food["total_pct_rdi"] / food["calories"]

In [2]:
for nutrient in micronutrients:
    display(
        food[
            (food["pct_rdi_per_calorie"] > 1.5)
            & (food["{}_grams_100_pct_rdi".format(nutrient)] < POUND_IN_GRAMS / 2)
        ][
            [
                "{}_calories_100_pct_rdi".format(nutrient),
                "{}_grams_100_pct_rdi".format(nutrient),
                "pct_rdi_per_calorie",
                "{}_pct_rdi".format(nutrient),
                "description",
                "category",
            ]
        ].sort_values(
            by=["{}_calories_100_pct_rdi".format(nutrient)], ascending=True
        )
    )

,calcium_calories_100_pct_rdi,calcium_grams_100_pct_rdi,pct_rdi_per_calorie,calcium_pct_rdi,description,category
4803,122.701383,105.777055,2.557567,94.538462,"TOFU,SALTED&FERMENTED (FUYU),PREP W/CA SULFATE",TOFU
7247,201.560874,135.275754,2.011097,73.923077,"CHEESE,MOZZARELLA,NON-FAT",CHEESE
7390,242.143600,135.275754,1.723064,73.923077,"CHEESE,SWISS,LOW FAT",CHEESE
4799,275.988287,190.336750,1.878023,52.538462,"TOFU,RAW,FIRM,PREP W/CA SULFATE",TOFU
7217,279.245283,188.679245,2.470633,53.000000,"CHEESE,PAST PROCESS,CHEDDAR OR AMERICAN,FAT-FREE",CHEESE
4801,287.535145,60.918463,1.766776,164.153846,"TOFU,DRIED-FROZEN (KOYADOFU),PREP W/CA SULFATE",TOFU
182,310.640216,117.222723,1.607726,85.307692,"CHEESE,PARMESAN,DRY GRATED,RED FAT",CHEESE
7142,313.625000,81.250000,4.481804,123.076923,"SMELT,DRIED (ALASKA NATIVE)",SMELT
7342,323.099415,190.058480,1.634445,52.615385,"CHEESE,PAST PROCESS,SWISS,LOFAT",CHEESE
7310,342.105263,190.058480,1.563648,52.615385,"CHEESE,PAST PROCESS,AMERICAN,LOFAT",CHEESE


,choline_calories_100_pct_rdi,choline_grams_100_pct_rdi,pct_rdi_per_calorie,choline_pct_rdi,description,category
3749,169.144603,112.016293,4.873988,89.272727,"BEEF,VAR MEATS&BY-PRODUCTS,BRAIN,CKD,SIMMRD",BEEF
3753,169.395712,107.212476,12.136184,93.272727,"BEEF,VAR MEATS&BY-PRODUCTS,KIDNEYS,CKD,SIMMRD",BEEF
3754,222.972973,165.165165,36.167017,60.545455,"BEEF,VAR MEATS&BY-PRODUCTS,LIVER,RAW",BEEF
3756,230.263158,131.578947,38.993059,76.000000,"BEEF, VAR MEATS&BY-PRODUCTS, LIVER, CKD, PAN-F...",BEEF
7118,230.645161,221.774194,26.979332,45.090909,"Fish, whitefish, eggs (Alaska Native)",Fish
4365,234.776119,164.179104,5.666650,60.909091,"ROE,MIXED SPECIES,RAW",ROE
114,244.354839,80.645161,1.927222,124.000000,"EGG,YOLK,RAW,FROZEN",EGG
3755,246.596244,129.107981,33.885779,77.454545,"BEEF,VAR MEATS & BY-PRODUCTS,LIVER,CKD,BRSD",BEEF
5071,248.387097,177.419355,41.843339,56.363636,"VEAL,VAR MEATS&BY-PRODUCTS,LIVER,RAW",VEAL
113,255.645161,80.645161,1.875900,124.000000,"EGG,YOLK,RAW,FRSH",EGG


,copper_calories_100_pct_rdi,copper_grams_100_pct_rdi,pct_rdi_per_calorie,copper_pct_rdi,description,category
5071,10.619469,7.585335,41.843339,1318.333333,"VEAL,VAR MEATS&BY-PRODUCTS,LIVER,RAW",VEAL
3756,10.796545,6.169454,38.993059,1620.888889,"BEEF, VAR MEATS&BY-PRODUCTS, LIVER, CKD, PAN-F...",BEEF
5073,11.541528,5.980066,40.363358,1672.222222,"VEAL,VAR MEATS&BY-PRODUCTS,LIVER,CKD,PAN-FRIED",VEAL
5072,11.566265,6.024096,43.451560,1660.000000,"VEAL,VAR MEATS&BY-PRODUCTS,LIVER,CKD,BRSD",VEAL
3755,12.035287,6.301197,33.885779,1587.000000,"BEEF,VAR MEATS & BY-PRODUCTS,LIVER,CKD,BRSD",BEEF
...,...,...,...,...,...,...
1637,767.250000,225.000000,3.529602,44.444444,"OSCAR MAYER,BRAUNSCHWEIGER LIVER SAUSAGE (SARE...",OSCAR MAYER
1161,1003.500000,225.000000,1.915691,44.444444,"SOUP,CRM OF VEG,DRY,PDR",SOUP
211,1029.512195,219.512195,1.842931,45.555556,"MUSTARD SEED,YELLOW",MUSTARD SEED
998,1039.500000,225.000000,1.676750,44.444444,"PATE DE FOIE GRAS,CND (GOOSE LIVER PATE),SMOKED",PATE DE FOIE GRAS


,iron_calories_100_pct_rdi,iron_grams_100_pct_rdi,pct_rdi_per_calorie,iron_pct_rdi,description,category
3762,42.424242,40.404040,7.985138,247.500000,"BEEF,VAR MEATS&BY-PRODUCTS,SPLEEN,RAW",BEEF
5083,43.399379,42.969683,6.964123,232.722222,"LAMB,VAR MEATS&BY-PRODUCTS,SPLEEN,RAW",LAMB
7060,58.260870,86.956522,2.099018,115.000000,"OOPAH (TUNICATE),WHL ANIMAL (ALASKA NATIVE)",OOPAH (TUNICATE)
3763,66.310976,45.731707,6.198816,218.666667,"BEEF,VAR MEATS&BY-PRODUCTS,SPLEEN,CKD,BRSD",BEEF
5084,72.614430,46.547711,4.597037,214.833333,"LAMB,VAR MEATS&BY-PRODUCTS,SPLEEN,CKD,BRSD",LAMB
...,...,...,...,...,...,...
4791,816.585366,219.512195,2.576677,45.555556,"SOY FLR,DEFATTED,CRUDE PROT BASIS (N X 6.25)",SOY FLR
4656,823.170732,219.512195,2.555619,45.555556,"SOY FLOUR,LOW-FAT",SOY FLOUR
211,845.891784,180.360721,1.842931,55.444444,"MUSTARD SEED,YELLOW",MUSTARD SEED
4801,873.175745,184.994861,1.766776,54.055556,"TOFU,DRIED-FROZEN (KOYADOFU),PREP W/CA SULFATE",TOFU


,magnesium_calories_100_pct_rdi,magnesium_grams_100_pct_rdi,pct_rdi_per_calorie,magnesium_pct_rdi,description,category
6179,148.477905,68.739771,6.722880,145.476190,"WHEAT BRAN,CRUDE",WHEAT BRAN
7411,151.200000,168.000000,3.431571,59.523810,"SNAIL,RAW",SNAIL
3297,166.909091,54.545455,2.816304,183.333333,"SEAWEED,AGAR,DRIED",SEAWEED
6162,169.935980,53.777209,6.747548,185.952381,"RICE BRAN,CRUDE",RICE BRAN
5848,191.903808,84.168337,4.509092,118.809524,"COCOA,DRY PDR,UNSWTND",COCOA
...,...,...,...,...,...,...
3279,701.489362,223.404255,9.984429,44.761905,"PEPPERS,SWT,GRN,FREEZE-DRIED",PEPPERS
3629,732.765957,111.702128,6.198752,89.523810,"BRAZILNUTS,DRIED,UNBLANCHED",BRAZILNUTS
4589,736.105263,221.052632,1.591836,45.238095,"BEANS,WHITE,MATURE SEEDS,RAW",BEANS
3614,754.707692,129.230769,1.958127,77.380952,"SUNFLOWER SD KRNLS,DRIED",SUNFLOWER SD KRNLS


...

,v_b6_calories_100_pct_rdi,v_b6_grams_100_pct_rdi,pct_rdi_per_calorie,v_b6_pct_rdi,description,category
2832,63.511831,211.706102,10.489906,47.235294,"BALSAM-PEAR (BITTER GOURD),LEAFY TIPS,RAW",BALSAM-PEAR (BITTER GOURD)
3321,71.578947,223.684211,12.841840,44.705882,"BALSAM-PEAR (BITTER GOURD),LEAFY TIPS,CKD,BLD,...",BALSAM-PEAR (BITTER GOURD)
2833,76.052632,223.684211,12.086438,44.705882,"BALSAM-PEAR (BITTER GOURD),LEAFY TIPS,CKD,BLD,...",BALSAM-PEAR (BITTER GOURD)
2973,90.666667,141.666667,6.424246,70.588235,"HORSERADISH-TREE LEAFY TIPS,RAW",HORSERADISH-TREE LEAFY TIPS
2974,109.795479,182.992465,7.021006,54.647059,"HORSERADISH-TREE,LEAFY TIPS,CKD,BLD,DRND,WO/SALT",HORSERADISH-TREE
...,...,...,...,...,...,...
3600,793.628088,221.066320,2.995940,45.235294,"COTTONSEED FLR,PART DEFATTED (GLANDLESS)",COTTONSEED FLR
3615,1230.597015,211.442786,1.894758,47.294118,"SUNFLOWER SD KRNLS,DRY RSTD,WO/SALT",SUNFLOWER SD KRNLS
3704,1230.597015,211.442786,1.894758,47.294118,"SUNFLOWER SD KRNLS,DRY RSTD,W/SALT",SUNFLOWER SD KRNLS
3616,1270.707071,214.646465,1.956049,46.588235,"SUNFLOWER SD KRNLS,OIL RSTD,WO/SALT",SUNFLOWER SD KRNLS


,v_c_calories_100_pct_rdi,v_c_grams_100_pct_rdi,pct_rdi_per_calorie,v_c_pct_rdi,description,category
2185,1.293750,5.625000,78.951044,1777.777778,"ACEROLA JUICE,RAW",ACEROLA JUICE
2184,1.716738,5.364807,59.549235,1864.000000,"ACEROLA,(WEST INDIAN CHERRY),RAW",ACEROLA
1196,13.235294,120.320856,11.253490,83.111111,"SAUCE,RTS,PEPPER OR HOT",SAUCE
3552,13.242507,49.046322,9.622871,203.888889,"PEPPERS,SWEET,YELLOW,RAW",PEPPERS
3434,13.684211,52.631579,10.485579,190.000000,"PEPPERS,SWT,RED,CKD,BLD,DRND,W/SALT",PEPPERS
...,...,...,...,...,...,...
6960,353.954496,97.508126,6.664061,102.555556,"Formulated bar, POWER BAR, chocolate",Formulated bar
2410,383.571429,214.285714,1.790510,46.666667,"PINEAPPLE JUC,FRZ CONC,UNSWTND,UNDIL",PINEAPPLE JUC
3686,528.548124,146.818923,1.797017,68.111111,"CHESTNUTS,JAPANESE,DRIED",CHESTNUTS
1922,666.857143,171.428571,2.489980,58.333333,"CEREALS READY TO EAT,CHARACTER CEREALS (MOVIES...",CEREALS READY TO EAT


,v_e_calories_100_pct_rdi,v_e_grams_100_pct_rdi,pct_rdi_per_calorie,v_e_pct_rdi,description,category
4682,205.208333,34.722222,2.047980,288.000000,"PEANUT BUTTER,SMOOTH,VIT & MINERAL FORT",PEANUT BUTTER
4683,205.902778,34.722222,2.124303,288.000000,"PEANUT BUTTER,CHUNKY,VITAMIN&MINERAL FORT",PEANUT BUTTER
3616,244.426094,41.288192,1.956049,242.200000,"SUNFLOWER SD KRNLS,OIL RSTD,WO/SALT",SUNFLOWER SD KRNLS
3705,244.426094,41.288192,1.956049,242.200000,"SUNFLOWER SD KRNLS,OIL RSTD,W/SALT",SUNFLOWER SD KRNLS
3614,263.617213,45.139934,1.958127,221.533333,"SUNFLOWER SD KRNLS,DRIED",SUNFLOWER SD KRNLS
208,288.845727,83.240844,4.933914,120.133333,"GINGER,GROUND",GINGER
4365,306.428571,214.285714,5.666650,46.666667,"ROE,MIXED SPECIES,RAW",ROE
3615,334.482759,57.471264,1.894758,174.000000,"SUNFLOWER SD KRNLS,DRY RSTD,WO/SALT",SUNFLOWER SD KRNLS
3704,334.482759,57.471264,1.894758,174.000000,"SUNFLOWER SD KRNLS,DRY RSTD,W/SALT",SUNFLOWER SD KRNLS
655,378.551136,71.022727,1.867631,140.800000,"MARGARINE,MARGARINE-LIKE VEG OIL SPRD,60% FAT,TUB",MARGARINE


,v_k_calories_100_pct_rdi,v_k_grams_100_pct_rdi,pct_rdi_per_calorie,v_k_pct_rdi,description,category
2818,2.421053,10.526316,50.923601,950.000000,"AMARANTH LEAVES,RAW",AMARANTH LEAVES
2917,2.746988,14.457831,46.450332,691.666667,"CHARD,SWISS,RAW",CHARD
2985,4.081633,13.605442,31.024155,735.000000,"KALE,FRZ,CKD,BLD,DRND,WO/SALT",KALE
3401,4.081633,13.605442,31.024155,735.000000,"KALE,FRZ,CKD,BLD,DRND,W/SALT",KALE
2983,4.112607,14.687882,31.817852,680.833333,"KALE,CKD,BLD,DRND,WO/SALT",KALE
...,...,...,...,...,...,...
3308,378.888889,111.111111,2.966855,90.000000,"CARROT,DEHYDRATED",CARROT
655,631.391905,118.460020,1.867631,84.416667,"MARGARINE,MARGARINE-LIKE VEG OIL SPRD,60% FAT,TUB",MARGARINE
662,631.391905,118.460020,1.868276,84.416667,"MARGARINE, VEG OIL SPRD,60% FAT,TUB/BOTTLE,UNS...",MARGARINE
4654,745.352113,169.014085,1.915051,59.166667,"SOY FLR,FULL-FAT,RSTD",SOY FLR


,zinc_calories_100_pct_rdi,zinc_grams_100_pct_rdi,pct_rdi_per_calorie,zinc_pct_rdi,description,category
4456,8.236978,12.113203,35.076721,825.545455,"OYSTER,EASTERN,WILD,RAW",OYSTER
4458,8.298001,6.056935,31.400978,1651.000000,"OYSTER,EASTERN,WILD,CKD,MOIST HEAT",OYSTER
4459,8.345245,12.094557,34.087288,826.818182,"OYSTER,EASTERN,CANNED",OYSTER
4533,10.760870,14.945652,33.965708,669.090909,"OYSTER,EASTERN,WILD,CKD,DRY HEAT",OYSTER
4534,17.114979,29.008439,22.416209,344.727273,"OYSTER,EASTERN,FARMED,RAW",OYSTER
...,...,...,...,...,...,...
121,1237.500000,208.333333,1.881760,48.000000,"EGG,WHOLE,DRIED",EGG
3616,1249.904031,211.132438,1.956049,47.363636,"SUNFLOWER SD KRNLS,OIL RSTD,WO/SALT",SUNFLOWER SD KRNLS
3705,1249.904031,211.132438,1.956049,47.363636,"SUNFLOWER SD KRNLS,OIL RSTD,W/SALT",SUNFLOWER SD KRNLS
3614,1284.800000,220.000000,1.958127,45.454545,"SUNFLOWER SD KRNLS,DRIED",SUNFLOWER SD KRNLS
